In [1]:
cd ../../

/Users/in-divye.singh/Documents/Projects/MIC_predictor


In [2]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### pIC50

In [3]:
results = pd.read_csv("results/knn_regressor_all_data_combination_separate_prop_pIC50.csv", sep=';')

In [4]:
results

,Data,Regressor,Best_parameters,MSE,MAE,MAPE
0,aa_freq,KNN,"{'metric': 'manhattan', 'n_neighbors': 10, 'we...",6.404887,1.783988,15.334217
1,dipep_freq,KNN,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei...",4.929638,1.471659,12.267070
2,avg_protvec,KNN,"{'metric': 'euclidean', 'n_neighbors': 2, 'wei...",6.498897,1.783781,15.057070
3,shannon_entropy,KNN,"{'metric': 'manhattan', 'n_neighbors': 11, 'we...",5.897072,1.730841,14.705748
4,residue_repeats,KNN,"{'metric': 'manhattan', 'n_neighbors': 6, 'wei...",5.808975,1.667210,13.841934
...,...,...,...,...,...,...
2051,aa_freq+dipep_freq+avg_protvec+residue_repeats...,KNN,"{'metric': 'manhattan', 'n_neighbors': 6, 'wei...",4.838845,1.542169,12.804184
2052,aa_freq+dipep_freq+shannon_entropy+residue_rep...,KNN,"{'metric': 'manhattan', 'n_neighbors': 7, 'wei...",5.150265,1.595945,13.396485
2053,aa_freq+avg_protvec+shannon_entropy+residue_re...,KNN,"{'metric': 'manhattan', 'n_neighbors': 7, 'wei...",5.001320,1.563774,13.091918
2054,dipep_freq+avg_protvec+shannon_entropy+residue...,KNN,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei...",5.286135,1.587492,13.081379


In [5]:
print(f"Best MAPE: {results['MAPE'].min()}")
print(f"Best performing data: {results.loc[results['MAPE'].argmin()]['Data']}")
print(f"Best parameters: {results.loc[results['MAPE'].argmin()]['Best_parameters']}")

Best MAPE: 11.932217912963191
Best performing data: dipep_freq+turn
Best parameters: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}


### MIC

In [6]:
results = pd.read_csv("results/knn_regressor_all_data_combination_separate_props_mic.csv", sep=';')

In [7]:
results[["PCC1", "p-value"]] = results.apply(lambda x:eval(x.PCC), axis=1, result_type='expand')

In [8]:
results = results.drop(['PCC'], axis=1)
results.rename(columns={'PCC1':'PCC'}, inplace=True)

In [9]:
results

,Data,Regressor,Best parameters,MSE,MAE,MAPE,PCC,p-value
0,aa_freq,KNN,"{'metric': 'chebyshev', 'n_neighbors': 1, 'wei...",4149.758538,32.125069,4.607827e+06,0.517361,3.689656e-11
1,dipep_freq,KNN,"{'metric': 'euclidean', 'n_neighbors': 1, 'wei...",3773.743816,28.484085,4.536615e+06,0.629510,3.730643e-17
2,avg_protvec,KNN,"{'metric': 'chebyshev', 'n_neighbors': 1, 'wei...",5264.466418,32.421390,1.443086e+06,0.559092,3.950182e-13
3,shannon_entropy,KNN,"{'metric': 'chebyshev', 'n_neighbors': 1, 'wei...",5181.276823,33.463097,3.216092e+06,0.356025,1.277457e-05
4,residue_repeats,KNN,"{'metric': 'euclidean', 'n_neighbors': 17, 'we...",2984.970463,30.865208,4.757993e+06,0.615487,2.845082e-16
...,...,...,...,...,...,...,...,...
2042,aa_freq+dipep_freq+avg_protvec+residue_repeats...,KNN,"{'metric': 'euclidean', 'n_neighbors': 2, 'wei...",4330.546950,32.405181,2.139660e+05,0.522489,2.183405e-11
2043,aa_freq+dipep_freq+shannon_entropy+residue_rep...,KNN,"{'metric': 'euclidean', 'n_neighbors': 2, 'wei...",4169.992771,32.288114,2.139206e+05,0.529978,9.987910e-12
2044,aa_freq+avg_protvec+shannon_entropy+residue_re...,KNN,"{'metric': 'euclidean', 'n_neighbors': 2, 'wei...",4341.027995,32.517268,2.139212e+05,0.521149,2.506282e-11
2045,dipep_freq+avg_protvec+shannon_entropy+residue...,KNN,"{'metric': 'euclidean', 'n_neighbors': 2, 'wei...",4353.697911,32.805415,2.139203e+05,0.518699,3.220343e-11


In [10]:
print(f"Best MAPE: {results['MAPE'].min()}")
print(f"Best performing data: {results.loc[results['MAPE'].argmin()]['Data']}")
print(f"Best parameters: {results.loc[results['MAPE'].argmin()]['Best parameters']}")

Best MAPE: 5067.804799150275
Best performing data: aa_freq+avg_protvec+helix+turn+sheet
Best parameters: {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}


In [11]:
print(f"Best MAPE: {results['PCC'].max()}")
print(f"Best performing data: {results.loc[results['PCC'].argmax()]['Data']}")
print(f"Best parameters: {results.loc[results['PCC'].argmax()]['Best parameters']}")

Best MAPE: 0.7542307945628506
Best performing data: aa_freq+residue_repeats+aromaticity+helix+turn+sheet
Best parameters: {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'}


### MIC to pMIC to MIC

In [13]:
results = pd.read_csv("results/knn_regressor_all_data_combination_separate_prop_mic_to_pIC50_to_mic.csv", sep=';')

In [14]:
results[["PCC1", "p-value"]] = results.apply(lambda x:eval(x.PCC), axis=1, result_type='expand')

In [15]:
results = results.drop(['PCC'], axis=1)
results.rename(columns={'PCC1':'PCC'}, inplace=True)

In [16]:
results

,Data,Regressor,Best parameters,MSE,MAE,MAPE,PCC,p-value
0,aa_freq,KNN,"{'metric': 'manhattan', 'n_neighbors': 8, 'wei...",3585.097994,30.960164,1.366652e+06,0.660239,2.970210e-19
1,dipep_freq,KNN,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei...",2979.231669,27.870760,2.591004e+06,0.667016,9.469039e-20
2,avg_protvec,KNN,"{'metric': 'euclidean', 'n_neighbors': 2, 'wei...",3406.098437,27.481173,2.982380e+04,0.621354,1.231256e-16
3,shannon_entropy,KNN,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei...",3520.775256,30.187419,8.077896e+05,0.567972,1.383567e-13
4,residue_repeats,KNN,"{'metric': 'manhattan', 'n_neighbors': 6, 'wei...",3619.804330,30.602137,4.273258e+05,0.570579,1.010777e-13
...,...,...,...,...,...,...,...,...
2042,aa_freq+dipep_freq+avg_protvec+residue_repeats...,KNN,"{'metric': 'manhattan', 'n_neighbors': 6, 'wei...",2884.604881,28.982488,7.730282e+05,0.691200,1.240702e-21
2043,aa_freq+dipep_freq+shannon_entropy+residue_rep...,KNN,"{'metric': 'manhattan', 'n_neighbors': 7, 'wei...",3564.990643,30.646342,6.426592e+05,0.593095,5.961039e-15
2044,aa_freq+avg_protvec+shannon_entropy+residue_re...,KNN,"{'metric': 'manhattan', 'n_neighbors': 7, 'wei...",2948.510530,29.239695,6.265631e+05,0.684304,4.456704e-21
2045,dipep_freq+avg_protvec+shannon_entropy+residue...,KNN,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei...",3054.427909,29.295361,1.402192e+06,0.648957,1.868353e-18


In [17]:
print(f"Best MAPE: {results['MAPE'].min()}")
print(f"Best performing data: {results.loc[results['MAPE'].argmin()]['Data']}")
print(f"Best parameters: {results.loc[results['MAPE'].argmin()]['Best parameters']}")

Best MAPE: 28160.760065785333
Best performing data: aa_freq+shannon_entropy+residue_repeats+helix+turn+sheet+gravy
Best parameters: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}


In [18]:
print(f"Best PCC: {results['PCC'].max()}")
print(f"Best performing data: {results.loc[results['PCC'].argmax()]['Data']}")
print(f"Best parameters: {results.loc[results['PCC'].argmax()]['Best parameters']}")

Best PCC: 0.7493382073088943
Best performing data: aa_freq+dipep_freq+aromaticity+gravy
Best parameters: {'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}


### MIC with contact energy

In [19]:
results = pd.read_csv("results/knn_regressor_with_contact_energy_all_data_combination_separate_props_mic.csv", sep=';')

In [20]:
results[["PCC1", "p-value"]] = results.apply(lambda x:eval(x.PCC), axis=1, result_type='expand')

In [21]:
results = results.drop(['PCC'], axis=1)
results.rename(columns={'PCC1':'PCC'}, inplace=True)

In [22]:
results

,Data,Regressor,Best parameters,MSE,MAE,MAPE,PCC,p-value
0,aa_freq,KNN,"{'metric': 'chebyshev', 'n_neighbors': 1, 'wei...",4238.627327,32.290370,4.607780e+06,0.499782,2.088325e-10
1,dipep_freq,KNN,"{'metric': 'euclidean', 'n_neighbors': 1, 'wei...",3578.824434,27.736770,4.536848e+06,0.649439,1.729853e-18
2,avg_protvec,KNN,"{'metric': 'chebyshev', 'n_neighbors': 1, 'wei...",5264.466418,32.421390,1.443086e+06,0.559092,3.950182e-13
3,shannon_entropy,KNN,"{'metric': 'chebyshev', 'n_neighbors': 2, 'wei...",4392.812154,31.049264,3.698783e+05,0.398091,8.489555e-07
4,residue_repeats,KNN,"{'metric': 'euclidean', 'n_neighbors': 22, 'we...",3117.354898,31.687504,3.897677e+06,0.602882,1.623929e-15
...,...,...,...,...,...,...,...,...
4090,aa_freq+dipep_freq+avg_protvec+residue_repeats...,KNN,"{'metric': 'manhattan', 'n_neighbors': 1, 'wei...",4075.833633,29.995766,1.251026e+07,0.497214,2.668302e-10
4091,aa_freq+dipep_freq+shannon_entropy+residue_rep...,KNN,"{'metric': 'manhattan', 'n_neighbors': 1, 'wei...",4075.982430,30.446794,1.251091e+07,0.495833,3.041777e-10
4092,aa_freq+avg_protvec+shannon_entropy+residue_re...,KNN,"{'metric': 'manhattan', 'n_neighbors': 1, 'wei...",4012.767152,29.695207,1.251026e+07,0.511214,6.840873e-11
4093,dipep_freq+avg_protvec+shannon_entropy+residue...,KNN,"{'metric': 'manhattan', 'n_neighbors': 1, 'wei...",4052.243658,29.998647,1.250957e+07,0.506072,1.135926e-10


In [23]:
print(f"Best MAPE: {results['MAPE'].min()}")
print(f"Best performing data: {results.loc[results['MAPE'].argmin()]['Data']}")
print(f"Best parameters: {results.loc[results['MAPE'].argmin()]['Best parameters']}")

Best MAPE: 5067.804799150274
Best performing data: aa_freq+avg_protvec+helix+turn+sheet
Best parameters: {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}


In [24]:
print(f"Best MAPE: {results['PCC'].max()}")
print(f"Best performing data: {results.loc[results['PCC'].argmax()]['Data']}")
print(f"Best parameters: {results.loc[results['PCC'].argmax()]['Best parameters']}")

Best MAPE: 0.7562366568843634
Best performing data: dipep_freq+aromaticity+turn+sheet
Best parameters: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}


In [33]:
contact_energy_res = results[results['Data'].apply(lambda s: 'contact_energy' in s)]

In [34]:
contact_energy_res['MAPE'].min()

5104320.45309828

### MIC with pfeature

In [3]:
results = pd.read_csv("results/knn_regressor_with_pfeature_all_data_combination_separate_props_mic_complete.csv", sep=';')

In [4]:
results[["PCC1", "p-value"]] = results.apply(lambda x:eval(x.PCC), axis=1, result_type='expand')

In [5]:
results = results.drop(['PCC'], axis=1)
results.rename(columns={'PCC1':'PCC'}, inplace=True)

In [6]:
results

,Data,Best_params,MSE,MAE,MAPE,PCC,p-value
0,aa_freq,"{'metric': 'chebyshev', 'n_neighbors': 1, 'wei...",4149.758538,32.125069,4.607827e+06,0.517361,3.689656e-11
1,dipep_freq,"{'metric': 'euclidean', 'n_neighbors': 1, 'wei...",3773.743816,28.484085,4.536615e+06,0.629510,3.730643e-17
2,avg_protvec,"{'metric': 'chebyshev', 'n_neighbors': 1, 'wei...",5264.466418,32.421390,1.443086e+06,0.559092,3.950182e-13
3,shannon_entropy,"{'metric': 'chebyshev', 'n_neighbors': 1, 'wei...",5181.276823,33.463097,3.216092e+06,0.356025,1.277457e-05
4,residue_repeats,"{'metric': 'euclidean', 'n_neighbors': 17, 'we...",2984.970463,30.865208,4.757993e+06,0.615487,2.845082e-16
...,...,...,...,...,...,...,...
16378,aa_freq+dipep_freq+avg_protvec+residue_repeats...,"{'metric': 'chebyshev', 'n_neighbors': 1, 'wei...",5963.553229,35.231467,8.329054e+04,0.326185,7.018083e-05
16379,aa_freq+dipep_freq+shannon_entropy+residue_rep...,"{'metric': 'chebyshev', 'n_neighbors': 1, 'wei...",5963.553229,35.231467,8.329054e+04,0.326185,7.018083e-05
16380,aa_freq+avg_protvec+shannon_entropy+residue_re...,"{'metric': 'chebyshev', 'n_neighbors': 1, 'wei...",5963.553229,35.231467,8.329054e+04,0.326185,7.018083e-05
16381,dipep_freq+avg_protvec+shannon_entropy+residue...,"{'metric': 'chebyshev', 'n_neighbors': 1, 'wei...",5963.553229,35.231467,8.329054e+04,0.326185,7.018083e-05


In [7]:
print(f"Best MAPE: {results['MAPE'].min()}")
print(f"Best performing data: {results.loc[results['MAPE'].idxmin()]['Data']}")
print(f"Best parameters: {results.loc[results['MAPE'].idxmin()]['Best_params']}")

Best MAPE: 5067.804799150275
Best performing data: aa_freq+avg_protvec+helix+turn+sheet
Best parameters: {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}


In [8]:
print(f"Best MAPE: {results['PCC'].max()}")
print(f"Best performing data: {results.loc[results['PCC'].idxmin()]['Data']}")
print(f"Best parameters: {results.loc[results['PCC'].idxmin()]['Best_params']}")

Best MAPE: 0.7542307945628506
Best performing data: gravy
Best parameters: {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}


In [9]:
pfeature_res = results[results['Data'].apply(lambda s: ('paac' in s) or ('cetd' in s))].reset_index(drop=True)

In [10]:
print(f"Best MAPE: {pfeature_res['MAPE'].min()}")
print(f"Best performing data: {pfeature_res.loc[results['MAPE'].idxmin()]['Data']}")
print(f"Best parameters: {pfeature_res.loc[results['MAPE'].idxmin()]['Best_params']}")

Best MAPE: 83290.53813743441
Best performing data: paac+aromaticity+turn+gravy+net_charge_at_pH7point4
Best parameters: {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}


### HIV CD-HIT filtered pIC50 to MIC with pfeature

In [3]:
results = pd.read_csv("results/knn_regressor_with_pfeature_HIV_cdhit_filtered_all_data_combination_separate_props_pmic_to_mic.csv", sep=';')

In [4]:
results[["PCC1", "p-value"]] = results.apply(lambda x:eval(x.PCC), axis=1, result_type='expand')

In [5]:
results = results.drop(['PCC'], axis=1)
results.rename(columns={'PCC1':'PCC'}, inplace=True)

In [6]:
results

,Data,Best_params,MSE,MAE,MAPE,PCC,p-value
0,aa_freq,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei...",4950.989808,44.553242,135.590804,0.593828,0.000429
1,dipep_freq,"{'metric': 'chebyshev', 'n_neighbors': 2, 'wei...",6915.114902,51.759740,1210.953237,-0.108488,0.561288
2,avg_protvec,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei...",6235.975310,48.530596,295.258022,0.086053,0.645310
3,shannon_entropy,"{'metric': 'manhattan', 'n_neighbors': 2, 'wei...",4903.118461,42.935697,121.265277,0.510930,0.003312
4,residue_repeats,"{'metric': 'manhattan', 'n_neighbors': 10, 'we...",5702.791176,46.050967,160.663981,0.270428,0.141189
...,...,...,...,...,...,...,...
16378,aa_freq+dipep_freq+avg_protvec+residue_repeats...,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei...",4761.530929,43.657524,291.821140,0.380885,0.034520
16379,aa_freq+dipep_freq+shannon_entropy+residue_rep...,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei...",4762.002072,43.641983,288.832183,0.381115,0.034401
16380,aa_freq+avg_protvec+shannon_entropy+residue_re...,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei...",4761.924440,43.641518,288.840450,0.381129,0.034394
16381,dipep_freq+avg_protvec+shannon_entropy+residue...,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei...",4761.950451,43.641465,288.838470,0.381125,0.034396


In [7]:
print(f"Best MAPE: {results['MAPE'].min()}")
print(f"Best performing data: {results.loc[results['MAPE'].idxmin()]['Data']}")
print(f"Best parameters: {results.loc[results['MAPE'].idxmin()]['Best_params']}")

Best MAPE: 102.41115959772229
Best performing data: aa_freq+dipep_freq+avg_protvec+turn+sheet
Best parameters: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}


In [8]:
print(f"Best MAPE: {results['PCC'].max()}")
print(f"Best performing data: {results.loc[results['PCC'].idxmin()]['Data']}")
print(f"Best parameters: {results.loc[results['PCC'].idxmin()]['Best_params']}")

Best MAPE: 0.6939574517697322
Best performing data: helix+sheet
Best parameters: {'metric': 'manhattan', 'n_neighbors': 10, 'weights': 'distance'}


In [10]:
results[results['Data'] == "aa_freq+avg_protvec+helix+turn+sheet"]

,Data,Best_params,MSE,MAE,MAPE,PCC,p-value
1845,aa_freq+avg_protvec+helix+turn+sheet,"{'metric': 'manhattan', 'n_neighbors': 4, 'wei...",5213.691023,45.903584,155.427353,0.517268,0.002883


In [12]:
results[results['Data'] == "aa_freq+dipep_freq+avg_protvec+helix+turn+sheet"]

,Data,Best_params,MSE,MAE,MAPE,PCC,p-value
3627,aa_freq+dipep_freq+avg_protvec+helix+turn+sheet,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei...",5041.680097,45.368645,134.184855,0.588722,0.000494
